In [62]:
import time
import requests
import json
from pprint import pprint
import pandas as pd
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

In [4]:
weather_url = 'https://api.met.no/weatherapi/locationforecast/2.0/compact?lat=69.64&lon=18.95'
weather_data = requests.get(weather_url).json()


In [9]:
pprint(weather_data)

{'geometry': {'coordinates': [18.95, 69.64, 0], 'type': 'Point'},
 'properties': {'meta': {'units': {'air_pressure_at_sea_level': 'hPa',
                                   'air_temperature': 'celsius',
                                   'cloud_area_fraction': '%',
                                   'precipitation_amount': 'mm',
                                   'relative_humidity': '%',
                                   'wind_from_direction': 'degrees',
                                   'wind_speed': 'm/s'},
                         'updated_at': '2022-08-18T08:39:33Z'},
                'timeseries': [{'data': {'instant': {'details': {'air_pressure_at_sea_level': 1019.7,
                                                                 'air_temperature': 11.7,
                                                                 'cloud_area_fraction': 71.9,
                                                                 'relative_humidity': 89.1,
                                         

In [11]:
weather_data['geometry']


{'type': 'Point', 'coordinates': [18.95, 69.64, 0]}

In [13]:
weather_data['properties']['meta']

{'updated_at': '2022-08-18T08:39:33Z',
 'units': {'air_pressure_at_sea_level': 'hPa',
  'air_temperature': 'celsius',
  'cloud_area_fraction': '%',
  'precipitation_amount': 'mm',
  'relative_humidity': '%',
  'wind_from_direction': 'degrees',
  'wind_speed': 'm/s'}}

In [16]:
weather_data['properties']['timeseries']

[{'time': '2022-08-18T08:00:00Z',
  'data': {'instant': {'details': {'air_pressure_at_sea_level': 1019.7,
     'air_temperature': 11.7,
     'cloud_area_fraction': 71.9,
     'relative_humidity': 89.1,
     'wind_from_direction': 198.7,
     'wind_speed': 4.0}},
   'next_12_hours': {'summary': {'symbol_code': 'fair_day'}},
   'next_1_hours': {'summary': {'symbol_code': 'partlycloudy_day'},
    'details': {'precipitation_amount': 0.0}},
   'next_6_hours': {'summary': {'symbol_code': 'partlycloudy_day'},
    'details': {'precipitation_amount': 0.0}}}},
 {'time': '2022-08-18T09:00:00Z',
  'data': {'instant': {'details': {'air_pressure_at_sea_level': 1020.4,
     'air_temperature': 12.1,
     'cloud_area_fraction': 95.7,
     'relative_humidity': 83.0,
     'wind_from_direction': 217.3,
     'wind_speed': 4.2}},
   'next_12_hours': {'summary': {'symbol_code': 'fair_day'}},
   'next_1_hours': {'summary': {'symbol_code': 'cloudy'},
    'details': {'precipitation_amount': 0.0}},
   'next_6_ho

In [45]:
weather_data['properties']['timeseries'][0]['data']['next_1_hours']['details']

{'precipitation_amount': 0.0}

In [60]:
data = [weather_data['properties']['timeseries'][i]['data']['instant']['details'] for i in range(len(weather_data['properties']['timeseries']))]
times = [weather_data['properties']['timeseries'][i]['time'] for i in range(len(weather_data['properties']['timeseries']))]
df = pd.DataFrame(data, index=times)
def precip1h(i):
    try: 
        return weather_data['properties']['timeseries'][i]['data']['next_1_hours']['details']
    except: 
        return {'precipitation_amount': None}
precipitation1h = pd.DataFrame([precip1h(i) for i in range(len(weather_data['properties']['timeseries']))], index=times)
precipitation1h
df = df.join(precipitation1h)


In [61]:
df

,air_pressure_at_sea_level,air_temperature,cloud_area_fraction,relative_humidity,wind_from_direction,wind_speed,precipitation_amount
2022-08-18T08:00:00Z,1019.7,11.7,71.9,89.1,198.7,4.0,0.0
2022-08-18T09:00:00Z,1020.4,12.1,95.7,83.0,217.3,4.2,0.0
2022-08-18T10:00:00Z,1021.1,11.8,66.5,87.0,222.8,3.3,0.0
2022-08-18T11:00:00Z,1021.3,13.4,32.1,75.5,215.2,3.2,0.0
2022-08-18T12:00:00Z,1021.3,14.1,20.5,67.8,220.7,2.5,0.0
...,...,...,...,...,...,...,...
2022-08-26T06:00:00Z,1015.5,11.1,52.0,86.1,199.9,4.8,NaN
2022-08-26T12:00:00Z,1018.9,12.4,100.0,81.9,238.0,5.1,NaN
2022-08-26T18:00:00Z,1016.2,10.5,100.0,93.3,233.0,6.7,NaN
2022-08-27T00:00:00Z,1018.7,9.2,100.0,95.6,243.9,6.2,NaN


In [ ]:
# needs this format
# _now = datetime.utcnow()
# _data_frame = pd.DataFrame(data=[["coyote_creek", 1.0], ["coyote_creek", 2.0]],
#                                    index=[_now, _now + timedelta(hours=1)],
#                                    columns=["location", "water_level"])
# 
# write_api.write("my-bucket", record=_data_frame, data_frame_measurement_name='h2o_feet',
#                             data_frame_tag_columns=['location'])

In [65]:
org = "FHSWF"
bucket = "weather"
token="clmwcDxm_39ML529Q5lbtT4XRLChNqqGull5VJt_zrFkAw4rTTPcMyzkhPo2sAlDhr6mIJW66en1PqRGmybRXQ=="
with InfluxDBClient("http://localhost:8086", token=token, org=org) as dbclient:
    write_api = dbclient.write_api(write_options=SYNCHRONOUS)
    write_api.write(bucket, record=df, data_frame_measurement_name='weather', data_frame_tag_columns='test')